In [28]:

bucket_name = 'hawaii-marine-debris'
file_path_template = "{}/jpg/{}"
downloaded_tile = set()
csv_rows = []
islands= ['niihau']#'lanai','bigisland',''
local_tmp_folder = "/tmp/{}"
local_tmp_image_folder = "/tmp/images/{island}/{filename}"
tile_with_debris = []


In [29]:
import os
import boto3 
s3_client = boto3.client('s3',
            aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
            aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
s3_resource = boto3.resource('s3',aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
            aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
    
s3_resource.Bucket(bucket_name).download_file("niihau/deduped_annotations.csv", "/tmp/niihau/annotations.csv")
def download_image_from_s3(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

In [30]:
import pandas as pd
df  =pd.read_csv("/tmp/niihau/annotations.csv")


In [32]:
df.groupby('label').count()

,s3_key,x0,y0,x1,y1
label,,,,,
B,1999,1999,1999,1999,1999
C,19,19,19,19,19
M,25,25,25,25,25
N,742,742,742,742,742
P,3658,3658,3658,3658,3658
T,306,306,306,306,306
V,12,12,12,12,12


In [33]:
def local_file_path(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    return local_file_path
df['local_file_path'] = df['s3_key'].apply(local_file_path)

In [39]:
import numpy as np
msk = np.random.rand(len(df)) < 0.9

train = df[msk]
test = df[~msk]

train[['local_file_path','x0','y0','x1','y1','label']].to_csv('keras_annotations_train.csv', encoding='utf-8', header=False, index=False)
test[['local_file_path','x0','y0','x1','y1','label']].to_csv('keras_annotations_test.csv', encoding='utf-8', header=False, index=False)


In [ ]:

for index, row in df[:20].iterrows():
    local_file = download_image_from_s3(row['s3_key'])
    print(local_file)